In [1]:
import urllib.request
import bz2
import os

url = "http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2"
response = urllib.request.urlopen(url)
compressed_data = response.read()

data = bz2.decompress(compressed_data)

save_path = "shape_predictor_68_face_landmarks.dat"
with open(save_path, 'wb') as f:
    f.write(data)

datFile = save_path

In [4]:
import cv2
import dlib
import numpy as np

detector = dlib.get_frontal_face_detector()

predictor = dlib.shape_predictor(datFile)

image1 = cv2.imread("face1.jpg")
image2 = cv2.imread("face2.jpg")

gray1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
gray2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)

faces1 = detector(gray1)
faces2 = detector(gray2)

landmarks1 = predictor(gray1, faces1[0])
landmarks2 = predictor(gray2, faces2[0])

face1_copy = image1.copy()
face2_copy = image2.copy()

points1 = []
points2 = []

for n in range(68):
    x1, y1 = landmarks1.part(n).x, landmarks1.part(n).y
    x2, y2 = landmarks2.part(n).x, landmarks2.part(n).y

    points1.append((x1, y1))
    points2.append((x2, y2))

points1 = np.array(points1, np.int32)
points2 = np.array(points2, np.int32)
rect1 = cv2.boundingRect(points1)
rect2 = cv2.boundingRect(points2)

x_offset = rect2[0] - rect1[0]
y_offset = rect2[1] - rect1[1]

image1_face = image1[rect1[1]:rect1[1] + rect1[3], rect1[0]:rect1[0] + rect1[2]]
image2_face = image2[rect2[1]:rect2[1] + rect2[3], rect2[0]:rect2[0] + rect2[2]]

result = cv2.seamlessClone(image2_face, image1, mask=np.ones_like(image1_face, dtype=np.uint8) * 255,
                            p=(rect1[0] + x_offset, rect1[1] + y_offset), flags=cv2.NORMAL_CLONE)

cv2.imshow("Face Swapped", result)
cv2.waitKey(0)
cv2.destroyAllWindows()